In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import keras
from hyperopt import STATUS_OK, Trials, fmin, hp, tpe
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
import mlflow
from mlflow.models import infer_signature

In [2]:
df=pd.read_csv(r"C:\Users\Garv Khurana\OneDrive\Desktop\mlops practise\wine-quality.csv",sep=',')
df.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.0010,3.00,0.45,8.8,6
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.9940,3.30,0.49,9.5,6
2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.9951,3.26,0.44,10.1,6
3,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6
4,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6


In [3]:
x=df.drop('quality',axis=1)
y=df['quality']
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.3,random_state=0)

In [4]:
x_train,x_val,y_train,y_val=train_test_split(x_train,y_train,test_size=0.2,random_state=0)

In [5]:
print(x_train.shape)
print(x_val.shape)
print(x_test.shape)

(2742, 11)
(686, 11)
(1470, 11)


In [6]:
mean=np.array(np.mean(x_train,axis=0))
variance=np.array(np.var(x_train,axis=0))

In [7]:
import numpy as np
x_train = np.array(x_train)  
y_train = np.array(y_train)
x_test = np.array(x_test)
y_test = np.array(y_test)
x_val = np.array(x_val)
y_val = np.array(y_val)

In [8]:
signature=infer_signature(x_train,y_train)

In [9]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Input,Normalization

In [10]:
def ann_model(x_train, y_train, x_test, y_test, x_val, y_val, params, epochs):
    model = Sequential()
    model.add(Input(shape=(x_train.shape[1],)))
    model.add(Normalization(mean=mean, variance=variance))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(1, activation='relu'))
    
    model.compile(optimizer=tf.keras.optimizers.SGD(learning_rate=params['lr'],
                                    momentum=params['momentum']),
                  loss='mean_squared_error',
                  metrics=[tf.keras.metrics.RootMeanSquaredError()])
    
    with mlflow.start_run(nested=True):
        model.fit(x_train, y_train, epochs=epochs,
                  validation_data=(x_val, y_val), batch_size=8)
        
        result = model.evaluate(x_test, y_test, batch_size=8)
        
        eval_rmse = result[1]
        
        mlflow.log_params(params)
        mlflow.log_metric('rmse', eval_rmse)
        mlflow.tensorflow.log_model(model, 'model', signature=signature)
    
    return {'loss': eval_rmse, 'status': STATUS_OK, 'model': model}
        

In [11]:
def objective(params):
    result=ann_model(x_train,
                     y_train,
                     x_test,
                     y_test,
                     x_val,
                     y_val,
                     params, 
                     epochs=3
                     )
    return result

In [12]:
space={'lr': hp.loguniform('lr', 0.01, 0.1),
         'momentum': hp.uniform('momentum', 0.1, 0.9)}
       

In [13]:
mlflow.set_experiment('wine__quality')
trials=Trials()
with mlflow.start_run():
    best=fmin(
        fn=objective,
        space=space,
        algo=tpe.suggest,
        max_evals=4,
        trials=trials
    )
    
    best_run=sorted(trials.results, key=lambda x: x['loss'])[0]
    print(best_run)

2025/01/18 19:54:55 INFO mlflow.tracking.fluent: Experiment with name 'wine__quality' does not exist. Creating a new experiment.


Epoch 1/3                                            

  1/343 ━━━━━━━━━━━━━━━━━━━━ 8:00 1s/step - loss: 34.5445 - root_mean_squared_error: 5.8775
 14/343 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 128.5806 - root_mean_squared_error: 10.9583
 29/343 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 94.8726 - root_mean_squared_error: 9.4051  
 45/343 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 79.3020 - root_mean_squared_error: 8.6016
 60/343 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 71.0494 - root_mean_squared_error: 8.1521
 74/343 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 65.9093 - root_mean_squared_error: 7.8630
 87/343 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 62.4373 - root_mean_squared_error: 7.6642
101/343 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 59.5458 - root_mean_squared_error: 7.4956
115/343 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 57.2543 - root_mean_squared_error: 7.3600
127/343 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 55.6430 - root_mean_squared_error: 7.2636
141/343 ━━━━━━━━━━━━━━━━━━━━ 0